#1. Огляд та попередній аналіз даних:
- Завантаження даних.
- Перевірка наявності пропусків, базова статистика.

In [1]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Download latest version
path = kagglehub.dataset_download("yasserh/titanic-dataset")

print("Path to dataset files:", path)
df = pd.read_csv(f'{path}/Titanic-Dataset.csv')

Path to dataset files: /kaggle/input/titanic-dataset


In [2]:
df.isna().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [3]:
missing = df.isnull()

fig = px.imshow(
    missing.T,
    color_continuous_scale="viridis",
    aspect="auto",
    labels=dict(x="Row Index", y="Column", color="Missing"),
    title="Missing Values Heatmap"
)

fig.update_xaxes(showticklabels=False)
fig.show()

print("Missing values per column:\n", df.isnull().sum())

Missing values per column:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
463,464,0,2,"Milling, Mr. Jacob Christian",male,48.0,0,0,234360,13.000,NaN,S
505,506,0,1,"Penasco y Castellana, Mr. Victor de Satode",male,18.0,1,0,PC 17758,108.900,C65,C
511,512,0,3,"Webber, Mr. James",male,NaN,0,0,SOTON/OQ 3101316,8.050,NaN,S
88,89,1,1,"Fortune, Miss. Mabel Helen",female,23.0,3,2,19950,263.000,C23 C25 C27,S
168,169,0,1,"Baumann, Mr. John D",male,NaN,0,0,PC 17318,25.925,NaN,S


# Візуалізація:


- Для акселерометра: побудувати графіки зміни значень по осях, розподіли активностей.
- Для фейкових новин: аналіз довжини текстів, частоти слів, wordcloud.
- Для Titanic Dataset: візуалізувати залежності між характеристиками та виживанням (наприклад, розподіли за віком і статтю, heatmap кореляцій, bar chart виживання по класах, статі тощо).

In [6]:
# Age vs Survived
fig1 = px.violin(
    df,
    x='Survived',
    y='Age',
    box=True,
    points='all',
    color='Survived',
    title='Age vs Survived'
)
fig1.update_layout(xaxis_title='Survived', yaxis_title='Age')
fig1.show()

In [7]:
# Survival by Class (grouped bar chart)
fig2 = px.histogram(
    df,
    x='Pclass',
    color='Survived',
    barmode='group',
    title='Survival by Class'
)
fig2.update_layout(
    xaxis_title='Passenger Class',
    yaxis_title='Count',
    legend_title='Survived'
)
fig2.show()

In [8]:
# Survival by Sex (grouped bar chart)
fig3 = px.histogram(
    df,
    x='Sex',
    color='Survived',
    barmode='group',
    title='Survival by Sex'
)
fig3.update_layout(
    xaxis_title='Sex',
    yaxis_title='Count',
    legend_title='Survived'
)
fig3.show()

In [9]:
# Correlation Matrix Heatmap
corr = df[['Age', 'Fare', 'Pclass', 'Survived']].corr().round(2)
fig4 = ff.create_annotated_heatmap(
    z=corr.values,
    x=list(corr.columns),
    y=list(corr.index),
    colorscale='RdBu',
    showscale=True,
    reversescale=True
)
fig4.update_layout(title='Correlation Matrix', width=500, height=450,
                   xaxis=dict(tickangle=45))
fig4.show()

# 3. Попередня обробка:

- Нормалізація/скейлінг (для числових).
- Токенізація/TF-IDF/ембедінги (для тексту).

In [10]:
# видалимо рядки із пропусками в цільовій змінній (Survived)
df = df.dropna(subset=['Survived'])

# видаляємо стовпці, які не використовуються
X = df.drop(columns=['Survived','Name','Ticket','Cabin'])
y = df['Survived']

# розбивка на train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

In [11]:
# побудова препроцесінг-пайплайну
numeric_features   = ['Age','Fare']
categorical_feats  = ['Pclass','Sex','Embarked']

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, numeric_features),
    ('cat', cat_pipeline, categorical_feats)
])

#4. Моделювання:


- Побудова базової моделі (Logistic Regression / Random Forest / SVM / інше).
- Порівняння з іншими моделями.


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# визначення моделей у пайплайнах
models = {
    'LogisticRegression': Pipeline([
        ('prep', preprocessor),
        ('clf', LogisticRegression(max_iter=500))
    ]),
    'RandomForest': Pipeline([
        ('prep', preprocessor),
        ('clf', RandomForestClassifier(n_estimators=100, random_state=42))
    ]),
    'SVM': Pipeline([
        ('prep', preprocessor),
        ('clf', SVC(kernel='rbf', probability=True))
    ])
}

In [13]:
# Cell 8: навчання моделей
for name, pipe in models.items():
    pipe.fit(X_train, y_train)
    print(f"{name} trained")

LogisticRegression trained
RandomForest trained
SVM trained


# 5. Оцінка моделі:


- Розділення на train/test.
- Метрики: accuracy, precision, recall, F1-score, confusion matrix.


In [15]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)
import plotly.express as px

# Оцінка на тестовій вибірці з Plotly
results = {}
for name, pipe in models.items():
    preds = pipe.predict(X_test)
    probs = pipe.predict_proba(X_test)[:, 1]

    # обчислюємо метрики
    acc   = accuracy_score(y_test, preds)
    prec  = precision_score(y_test, preds)
    rec   = recall_score(y_test, preds)
    f1    = f1_score(y_test, preds)
    auc   = roc_auc_score(y_test, probs)

    results[name] = {
        'accuracy':  acc,
        'precision': prec,
        'recall':    rec,
        'f1':        f1,
        'roc_auc':   auc
    }

    # вивід метрик у консоль
    print(f"\n=== {name} ===")
    print(f"Accuracy : {acc:.3f}")
    print(f"Precision: {prec:.3f}")
    print(f"Recall   : {rec:.3f}")
    print(f"F1-score : {f1:.3f}")
    print(f"ROC AUC  : {auc:.3f}")

    # будуємо confusion matrix
    cm = confusion_matrix(y_test, preds)

    # відображаємо heatmap через Plotly
    fig = px.imshow(
        cm,
        labels=dict(x="Predicted", y="Actual", color="Count"),
        x=["Survived=0", "Survived=1"],
        y=["True=0",      "True=1"],
        text_auto=True,
        color_continuous_scale="Blues",
        title=f"Confusion Matrix — {name}"
    )
    fig.update_layout(width=500, height=450)
    fig.show()


=== LogisticRegression ===
Accuracy : 0.777
Precision: 0.730
Recall   : 0.667
F1-score : 0.697
ROC AUC  : 0.834



=== RandomForest ===
Accuracy : 0.816
Precision: 0.773
Recall   : 0.739
F1-score : 0.756
ROC AUC  : 0.829



=== SVM ===
Accuracy : 0.810
Precision: 0.872
Recall   : 0.594
F1-score : 0.707
ROC AUC  : 0.824


# Висновки:


Що вдалося? Що можна покращити?